In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image

import pathlib
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!ls ../input/plant-seedlings-classification

In [ ]:
data_dir = '../input/plant-seedlings-classification/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [ ]:
train_dir = pathlib.Path(train_dir)
image_count = len(list(train_dir.glob('*/*.png')))
print(image_count)

In [ ]:
batch_size = 32
img_height = 260
img_width = 260

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                              seed = 7,
                                                              image_size = (img_height, img_width),
                                                              batch_size = batch_size)

'''val_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                            validation_split = 0.2,
                                                            subset = "validation",
                                                            seed = 7,
                                                            image_size = (img_height, img_width),
                                                            batch_size = batch_size)'''

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)
print(num_classes)
#print(type(class_names))

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [ ]:
plt.figure(figsize=(10, 10))
for image, label in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        aug_img = img_augmentation(image)
        plt.imshow(aug_img[i].numpy().astype("uint8"))
        plt.title("{}".format(class_names[label[i]]))
        plt.axis("off")

In [ ]:
def input_preprocess(image, label):
    label = tf.one_hot(label, 12)
    return image, label

In [ ]:
train_ds = train_ds.map(
    input_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE
)

#val_ds = val_ds.map(input_preprocess)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
from tensorflow.keras.applications import EfficientNetB2


def build_model(num_classes):
    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB2(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dropout(0.3, name = 'drop2')(x)
    x = layers.Dense(100, activation = "relu")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dropout(0.2, name = 'drop1')(x)
    x = layers.Dense(50, activation = "relu")(x)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
with strategy.scope():
    model = build_model(num_classes=num_classes)

model.summary()

In [ ]:
epochs = 30  # @param {type: "slider", min:8, max:80}
hist = model.fit(train_ds, epochs=epochs, verbose=2)

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
    plt.plot(hist.history['loss'], label='train')
    # plt.plot(result.history['val_loss'], label='test')
    plt.legend(loc='upper right')
    plt.title('Model Cost')
    plt.ylabel('Cost')
    plt.xlabel('Epoch')
    plt.show()

plot_hist(hist)

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 35 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-35:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

model.summary()

In [ ]:
epochs = 30  # @param {type: "slider", min:8, max:50}
hist = model.fit(train_ds, epochs=epochs, verbose=2)
plot_hist(hist)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

epochs = 20  # @param {type: "slider", min:8, max:50}
hist = model.fit(train_ds, epochs=epochs, verbose=2)
plot_hist(hist)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)

epochs = 10  # @param {type: "slider", min:8, max:50}
hist = model.fit(train_ds, epochs=epochs, verbose=2)
plot_hist(hist)

In [ ]:
test = []
for file in os.listdir(test_dir):
    test.append(['test/{}'.format(file), file])
test = pd.DataFrame(test, columns=['filepath', 'file'])
print(test.head(2))
test.shape

In [ ]:
from tqdm import tqdm

x_test = np.zeros((len(test), img_height, img_width, 3), dtype='float32')
for i, filepath in tqdm(enumerate(test['filepath'])):
    img = keras.preprocessing.image.load_img(os.path.join(data_dir, filepath), target_size = (img_height, img_width))
    x = keras.preprocessing.image.img_to_array(img)
    x_test[i] = x
print('test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))

In [ ]:
one_hot_prediction = model.predict(x_test)
one_hot_prediction[0:2]

In [ ]:
preds = np.argmax(one_hot_prediction, axis=1)
print(preds.shape)
print(preds)
prediction = [class_names[i] for i in preds]
print(len(prediction))
print(prediction)

In [ ]:
sample_submission.head(5)

In [ ]:
my_submission = pd.DataFrame({'file': test['file'], 'species': prediction})
print(my_submission.head(5))
print(my_submission.shape)

In [ ]:
my_submission.to_csv('output.csv', index=False)